In [1]:
pip install firecrawl-py python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
from firecrawl import FirecrawlApp
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = "fc-54426a76940c4642b6321da1902131fe"

# Initialize Firecrawl application
# app = FirecrawlApp(api_key=api_key)

# # Define the target URL
# url = "https://www.cbinsights.com/research/startup-failure-post-mortem/"

# # Scrape the URL
# response = app.scrape_url(url)

# # Extract and print the main content in Markdown format
# markdown_content = response.get('markdown')
# print(markdown_content)

In [7]:
# Save the Markdown content to a file,
# create the output directory if it doesn't exist
# output_dir = "output"
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)
# output_file = "output/output.md"
# with open(output_file, "w") as file:
#     file.write(markdown_content)
# print(f"Markdown content saved to {output_file}")

In [8]:
file_path = "./output/output.md"

try:
    with open(file_path, "r") as file_variable:
        content = file_variable.read()
        print(content)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")



### **Olive**

[Olive](https://www.cbinsights.com/company/crosschx) offered healthcare AI solutions that automated key tasks like prior authorization approvals and clinical analysis for surgeries. It consistently drew fresh capital over its lifetime, raising a new round nearly every year following its founding in 2012. However, the combination of challenging economic conditions, evolving customer expectations, and management missteps led the company to sell off its core assets and shut down.

- **Select VC investors:** Silicon Valley Bank, Google Ventures, Sequoia Capital, Tiger Global Management
- **Total equity funding:** $850M
- **Peak valuation:** $4B

[![CBI customers can click this banner to access instant insights on Olive's funding history](https://research-assets.cbinsights.com/2024/05/28183836/olive-banner-1024x77.png)](https://app.cbinsights.com/profiles/c/ven3M/fundings)

#### **Instant Insights: Olive exits market, sells off main business units in October 2023**

- Olive

In [9]:
# spllit the content by 30 h3 headings
def split_by_h3_headings(markdown_content, headings_per_chunk=30):
    """
    Splits markdown content into chunks where each chunk contains up to 
    the specified number of h3 headings.
    
    Args:
        markdown_content (str): The markdown content to split
        headings_per_chunk (int, optional): Number of h3 headings per chunk. Defaults to 30.
    
    Returns:
        list: A list of markdown chunks as strings
    """
    # Split the content by lines
    lines = markdown_content.split('\n')
    
    chunks = []
    current_chunk = []
    h3_count = 0
    
    for line in lines:
        # Check if the line is an h3 heading (starts with ###)
        is_h3 = line.lstrip().startswith('### ')
        
        # If we hit an h3 heading and already have the max h3s in the current chunk,
        # save the current chunk and start a new one
        if is_h3:
            h3_count += 1
            
            if h3_count > headings_per_chunk and h3_count % headings_per_chunk == 1:
                # Save current chunk when we hit the 31st, 61st, etc. h3 heading
                chunks.append('\n'.join(current_chunk))
                current_chunk = []
        
        # Add the current line to the current chunk
        current_chunk.append(line)
    
    # Add the last chunk if it has content
    if current_chunk:
        chunks.append('\n'.join(current_chunk))
    
    return chunks



In [10]:
split_content=split_by_h3_headings(content, headings_per_chunk=30)

In [11]:
split_content[0]

"\n\n### **Olive**\n\n[Olive](https://www.cbinsights.com/company/crosschx) offered healthcare AI solutions that automated key tasks like prior authorization approvals and clinical analysis for surgeries. It consistently drew fresh capital over its lifetime, raising a new round nearly every year following its founding in 2012. However, the combination of challenging economic conditions, evolving customer expectations, and management missteps led the company to sell off its core assets and shut down.\n\n- **Select VC investors:** Silicon Valley Bank, Google Ventures, Sequoia Capital, Tiger Global Management\n- **Total equity funding:** $850M\n- **Peak valuation:** $4B\n\n[![CBI customers can click this banner to access instant insights on Olive's funding history](https://research-assets.cbinsights.com/2024/05/28183836/olive-banner-1024x77.png)](https://app.cbinsights.com/profiles/c/ven3M/fundings)\n\n#### **Instant Insights: Olive exits market, sells off main business units in October 20

In [12]:
template_string = """As a professional summarizer, create a concise and comprehensive summary of the information for each failed startups in this webpadge, while adhering to these guidelines:\

**All summaries must be about healthcare startups, exclude startups that is not in the healthcare filed
**scan through the content and generate a summary for each startup failure, focusing on the key points and essential information.

1. **Detailed and Clear**  
   Craft a summary that is thorough, in-depth, and complex, ensuring clarity and precision in the information presented.

2. **Focus on Main Ideas**  
   Incorporate key points and essential information, ensure to cover all information includeing the context, emphasizing on critical aspects。

3. **Strictly Text-Based**  
   Rely exclusively on the provided text without adding external information.

4. **Paragraph Formatting**  
   Format the summary of each key point into a clear and readable paragraph.

Webpadge: ```{Webpadge}```

{format_instructions}
"""

In [ ]:
from langchain.output_parsers import PydanticOutputParser 
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser 
from pydantic import BaseModel, Field, field_validator
from typing import List
from langchain_openai import ChatOpenAI
from openai import OpenAI


import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

#llm = ChatOpenAI(model="o3-mini")
llm = OpenAI(api_key="OPENAI_API_KEY")


TypeError: OpenAI.__init__() got an unexpected keyword argument 'model'

In [14]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [15]:
class startup_data(BaseModel):
    startup_name: str = Field(description="The name of the startup")
    startup_description: str = Field(description="The description of the startup")
    startup_vc: str = Field(description="The name of the VC investors of the startup")
    startup_funding: str = Field(description="The funding amount of the startup")
    startup_peak_value: str = Field(description="The peak value of the startup")
    startup_failure_reason: str = Field(description="The reason for the startup's failure, make sure the reason is comperhensive and clear")
    # You can add custom validation logic easily with Pydantic.
pydantic_parser = PydanticOutputParser(pydantic_object=startup_data)

format_instructions = pydantic_parser.get_format_instructions()

In [16]:
prompt = ChatPromptTemplate.from_template(template=template_string)

messages = prompt.format_messages(Webpadge=split_content[0],
                                format_instructions=format_instructions)

In [17]:
print(messages[0].content)

As a professional summarizer, create a concise and comprehensive summary of the information for each failed startups in this webpadge, while adhering to these guidelines:
**All summaries must be about healthcare startups, exclude startups that is not in the healthcare filed
**scan through the content and generate a summary for each startup failure, focusing on the key points and essential information.

1. **Detailed and Clear**  
   Craft a summary that is thorough, in-depth, and complex, ensuring clarity and precision in the information presented.

2. **Focus on Main Ideas**  
   Incorporate key points and essential information, ensure to cover all information includeing the context, emphasizing on critical aspects。

3. **Strictly Text-Based**  
   Rely exclusively on the provided text without adding external information.

4. **Paragraph Formatting**  
   Format the summary of each key point into a clear and readable paragraph.

Webpadge: ```

### **Olive**

[Olive](https://www.cbinsi

In [18]:
# output=llm(messages)

output=llm.invoke(messages)
print(output.content)

# print(json_output)


[
  {
    "startup_name": "Olive",
    "startup_description": "Olive offered healthcare AI solutions that automated essential tasks such as prior authorization approvals and clinical analysis for surgeries. Founded in 2012, it successfully raised fresh capital repeatedly until challenging economic conditions, shifting customer expectations, and management missteps forced a strategic sale of its core assets.",
    "startup_vc": "Silicon Valley Bank, Google Ventures, Sequoia Capital, Tiger Global Management",
    "startup_funding": "$850M",
    "startup_peak_value": "$4B",
    "startup_failure_reason": "The combination of economic downturn, evolving customer demands, and internal management errors led to its inability to sustain operations, resulting in the sale of its key business units and a complete shutdown."
  },
  {
    "startup_name": "Avail",
    "startup_description": "Avail developed telepresence technologies enabling clinicians to communicate with external parties, a solution 

In [19]:
#use the json parser to parse the output


from langchain_core.output_parsers import JsonOutputParser
jsonoutput = JsonOutputParser()

pasred_output = jsonoutput.parse(output.content)

pasred_output



[{'startup_name': 'Olive',
  'startup_description': 'Olive offered healthcare AI solutions that automated essential tasks such as prior authorization approvals and clinical analysis for surgeries. Founded in 2012, it successfully raised fresh capital repeatedly until challenging economic conditions, shifting customer expectations, and management missteps forced a strategic sale of its core assets.',
  'startup_vc': 'Silicon Valley Bank, Google Ventures, Sequoia Capital, Tiger Global Management',
  'startup_funding': '$850M',
  'startup_peak_value': '$4B',
  'startup_failure_reason': 'The combination of economic downturn, evolving customer demands, and internal management errors led to its inability to sustain operations, resulting in the sale of its key business units and a complete shutdown.'},
 {'startup_name': 'Avail',
  'startup_description': 'Avail developed telepresence technologies enabling clinicians to communicate with external parties, a solution that rapidly gained traction 

In [21]:
import os
from openai import OpenAI

# Specify your OpenAI API key and embedding model
# os.environ["OPENAI_API_KEY"] 
model = "text-embedding-3-small"
openai_client = OpenAI()

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
import json
import os

chunk_docs = []

for startup in pasred_output:
    for key in startup:
        print(key)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        if key == "startup_name":
            # Create a chunk document for the startup name
            chunks = text_splitter.create_documents(["Startup_name : "+str(startup[key])], metadatas=[{"Startup_name": startup["startup_name"]}])
        else:
            chunks = text_splitter.create_documents([str("startup_name : "+startup['startup_name']+', '+key+" : "+startup[key])], metadatas=[{"Startup_name": startup["startup_name"]}])
        chunk_docs.append(chunks)

# Print the first chunk document


startup_name
startup_description
startup_vc
startup_funding
startup_peak_value
startup_failure_reason
startup_name
startup_description
startup_vc
startup_funding
startup_peak_value
startup_failure_reason
startup_name
startup_description
startup_vc
startup_funding
startup_peak_value
startup_failure_reason
startup_name
startup_description
startup_vc
startup_funding
startup_peak_value
startup_failure_reason
startup_name
startup_description
startup_vc
startup_funding
startup_peak_value
startup_failure_reason
startup_name
startup_description
startup_vc
startup_funding
startup_peak_value
startup_failure_reason
startup_name
startup_description
startup_vc
startup_funding
startup_peak_value
startup_failure_reason
startup_name
startup_description
startup_vc
startup_funding
startup_peak_value
startup_failure_reason
startup_name
startup_description
startup_vc
startup_funding
startup_peak_value
startup_failure_reason


In [22]:
chunk_docs

[[Document(metadata={'Startup_name': 'Olive'}, page_content='Startup_name : Olive')],
 [Document(metadata={'Startup_name': 'Olive'}, page_content='startup_name : Olive, startup_description : Olive offered healthcare AI solutions that automated essential tasks such as prior authorization approvals and clinical analysis for surgeries. Founded in 2012, it successfully raised fresh capital repeatedly until challenging economic conditions, shifting customer expectations, and management missteps forced a strategic sale of its core assets.')],
 [Document(metadata={'Startup_name': 'Olive'}, page_content='startup_name : Olive, startup_vc : Silicon Valley Bank, Google Ventures, Sequoia Capital, Tiger Global Management')],
 [Document(metadata={'Startup_name': 'Olive'}, page_content='startup_name : Olive, startup_funding : $850M')],
 [Document(metadata={'Startup_name': 'Olive'}, page_content='startup_name : Olive, startup_peak_value : $4B')],
 [Document(metadata={'Startup_name': 'Olive'}, page_con

In [26]:
chunk_docs[0][0].page_content

'Startup_name : Olive'

In [5]:
# import basics
import os
from dotenv import load_dotenv

# import langchain
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings

# import supabase
from supabase.client import Client, create_client

# load environment variables
load_dotenv()  

# initiate supabase db
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

# initiate embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")



In [35]:
chunk_docs

[[Document(metadata={'Startup_name': 'Olive'}, page_content='Startup_name : Olive')],
 [Document(metadata={'Startup_name': 'Olive'}, page_content='startup_name : Olive, startup_description : Olive offered healthcare AI solutions that automated essential tasks such as prior authorization approvals and clinical analysis for surgeries. Founded in 2012, it successfully raised fresh capital repeatedly until challenging economic conditions, shifting customer expectations, and management missteps forced a strategic sale of its core assets.')],
 [Document(metadata={'Startup_name': 'Olive'}, page_content='startup_name : Olive, startup_vc : Silicon Valley Bank, Google Ventures, Sequoia Capital, Tiger Global Management')],
 [Document(metadata={'Startup_name': 'Olive'}, page_content='startup_name : Olive, startup_funding : $850M')],
 [Document(metadata={'Startup_name': 'Olive'}, page_content='startup_name : Olive, startup_peak_value : $4B')],
 [Document(metadata={'Startup_name': 'Olive'}, page_con

In [34]:


# store chunks in vector store
# create a vector store
vector_store = SupabaseVectorStore(
    client=supabase,
    embedding=embeddings,
    table_name="documents",
    query_name="match_documents",
    chunk_size=1000,
)

# add documents to the vector store
for chunk in chunk_docs:
    vector_store.add_documents(chunk)

In [45]:
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document

# Load CSV
df = pd.read_csv("./csv/google_play_reviews_clean.csv")

# Set your embedding model
embeddings = OpenAIEmbeddings(openai_api_key="your-api-key")

# Convert each row to a Document (row as a chunk)
review_documents = []
for i, row in df.iterrows():
    # You can customize which columns to include
    content = " | ".join([f"{col}: {row[col]}" for col in df.columns])
    review_documents.append(Document(page_content=content, metadata={"row_index": i}))

# Vectorize

/var/folders/hn/262hwq1n5yq8003zjf198dt40000gn/T/ipykernel_23742/231127587.py:9: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key="your-api-key")


In [46]:
review_documents

[Document(metadata={'row_index': 0}, page_content='Product: Digital Wellbeing | Company: Google LLC | year: 2021 | review_id: 7a7b7f64-68fa-4222-954a-de6aa53e3ec6 | review_title: nan | user_name: A Google user | review_description: Scammm app | rating: 1 | thumbs_up: 0 | review_date: 7/7/21 21:52 | developer_response: nan | developer_response_date: nan | appVersion: 1.0.334333145'),
 Document(metadata={'row_index': 1}, page_content='Product: Digital Wellbeing | Company: Google LLC | year: 2024 | review_id: b3fe9a21-26b3-44e7-8d30-3f62ed706464 | review_title: nan | user_name: A Google user | review_description: Ok google | rating: 5 | thumbs_up: 0 | review_date: 7/7/21 21:13 | developer_response: nan | developer_response_date: nan | appVersion: 1.0.372116326'),
 Document(metadata={'row_index': 2}, page_content='Product: Digital Wellbeing | Company: Google LLC | year: 2025 | review_id: db4332b8-8a43-421b-93e6-024e627d773a | review_title: nan | user_name: A Google user | review_descriptio

In [54]:
vector_store_review = SupabaseVectorStore(
    client=supabase,
    embedding=embeddings,
    table_name="reviews_embeddings",
    query_name="match_review_documents",
    chunk_size=1000,
)

# add documents to the vector store
# for chunk in review_documents:
#     vector_store_review.add_documents(chunk)

vector_store_review.add_documents(review_documents)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-api-key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}